In [167]:
import os
import sys
import re
import pandas as pd
import pprint
import csv
import platform
import string


In [168]:
hostname = platform.node()
print(hostname)

if hostname == "T_800_Model_101":
   directory = "D:/Dropbox/"

if hostname == "ECON-STAFF-ASSOC-2022IMAC-H4TJ80ZFQ6X9-2":
   directory = "/Users/gr2757/Dropbox/"

ECON-STAFF-ASSOC-2022IMAC-H4TJ80ZFQ6X9-2


In [169]:
#making the case for asteriks as footnotes
def asterisk_case_func(line_by_line):
    #this is to make the asterik to be "(z)" to treat it as a footnote

    # Iterate through each line and replace "*" with "(z)"
    for i, line in enumerate(line_by_line):
        if "*" in line:
            asterisk_value = re.sub(r'\*', '(u)', line)
            line_by_line[i] = asterisk_value

    return(line_by_line)


In [170]:
#function to count leading whitespace
def white_space_count(line):
    length = len(line)- len(line.lstrip())
    return(int(length))

In [171]:
def remove_leading_whitespace(s, num_whitespace):
    # Find the count of leading whitespace
    leading_whitespace_count = len(s) - len(s.lstrip())
    
    # Remove the minimum of num_whitespace or leading_whitespace_count
    remove_count = min(num_whitespace, leading_whitespace_count)
    return s[remove_count:]

In [172]:
#get table function
def get_table(text_directory):
    with open(text_directory, "r") as file:
        text = file.read()
        file_name = os.path.basename(text_directory).split('/')[-1] #Need the file name to take out the country name and year of the data
        country_name = file_name[:-9] #Name of the country
        data_year = file_name[-8:-4] #What year is the data

        # combined_pattern =  r'(?i)^\s*A\.\s*At a glance\*?\s*(.*?)(?=^\s*B\.\s*)' #this one also works very well #most recent one that mighit work
        # combined_pattern = r'(?i)^\s*A\.\s*At a glance\*?\s*(.*?)(?=^\s*B\.\s*\w+.*$)' last one
        combined_pattern = r'(?i)^\s*A\.\s*At a glance\*?\s*(.*?)(?<!Section\s)(?=^\s*B\.\s*\w+.*$)'

    
        table_match = re.search(combined_pattern, text, re.DOTALL | re.MULTILINE)

        if table_match:
            table_text = table_match.group(1)
            # print(table_text)
        else:
            print("missing table", country_name, data_year)
        
        #this is to keep the easy tax rates and store them in a list for further processing. 
        line_by_line = table_text.split("\n")
        
        for i in range(len(line_by_line) - 1, -1, -1):
            if re.search(r"WWCT", line_by_line[i]):
                line_by_line.pop(i)
        
        #clearing empty list values due to problems in text processing
        i = 0
        while i < len(line_by_line):
            if line_by_line[i] == "":
                line_by_line.pop(i)
            else:
                i += 1

        ###removeing headers and page numbers:
                
        #these are all countries that I had to standardize their file names, so the country name is not the same as the header
        index_remove = []
        list_of_names = None
        if country_name == "South Korea":
            list_of_names = ["Korea", "Korea (South)", "South Korea"]

        if country_name == "Palestine":
            list_of_names = ["Palestinian Authority"]

        if country_name == "North Macedonia":
            list_of_names = ["North Macedonia", "Macedonia"]

        if country_name == "Côte D’Ivoire (Ivory Coast)":
            list_of_names =["Côte d’Ivoire", "Côte D’Ivoire(Ivory Coast)"]

        if country_name == "Guernsey, Channel Islands":
            list_of_names = ["Guernsey"]

        if country_name == "Congo, Republic of":
            list_of_names = ["Congo", "Congo, Republic of"]
        if country_name == "Jersey, Channel Islands":
            list_of_names = ["Jersey"]

        if list_of_names:
            for name in list_of_names:
                no_space_country = re.sub(r'\s+', '', name) #remove spaces
                no_space_country = no_space_country.upper() #make upper case
                for i in range(len(line_by_line)):
                    line = line_by_line[i] 
                    line =  re.sub(r'\s+', '', line)
                    line = re.sub(r'\d', "", line)
                    line = line.upper()

                    no_space = line.strip()
                    if no_space == no_space_country:
                        index_remove.append(i)
                        # print("fixed change country name", country_name, data_year)
                        break
        else: #if there are no list of names, then the country_name will do fine
            no_space_country = re.sub(r'\s+', '', country_name)
            no_space_country = no_space_country.upper()
            for i in range(len(line_by_line)):
                line = line_by_line[i]
                line =  re.sub(r'\s+', '', line)
                line = re.sub(r'\d', "", line)
                line = line.upper()
                no_space = line.strip()
                if no_space == no_space_country:
                    index_remove.append(i)

        #turning unicode fraction into 1/3
        for i in range(len(line_by_line)):
            if re.search(r"\u2153", line_by_line[i]):
                line_by_line[i] = re.sub(r"\u2153", "1/3", line_by_line[i])
        
        #remove        
        if index_remove:
            line_by_line = [item for i, item in enumerate(line_by_line) if i not in index_remove]

        #turning asterisks into (z)
        line_by_line = asterisk_case_func(line_by_line)
        ## in 2010 they have a lot of extra leading whitespace so this is code to standardize it to the rest of the years



        standard_level = None

        if data_year == "2010":
            for i in range(len(line_by_line)):
                if re.search(r"\s*Net Operating Losses \(Years\)\s*", line_by_line[i]):
                    standard_level = white_space_count(line_by_line[i])

            for k in range(len(line_by_line)):
                if re.search(r"\s*Withholding\s*", line_by_line[k]):
                    if standard_level:
                        if white_space_count(line_by_line[k]) < standard_level:
                            standard_level = white_space_count(line_by_line[k])
                    else:
                        standard_level = white_space_count(line_by_line[k])


            for i in range(len(line_by_line)):
                if country_name == "Portugal" and re.search(r'\b(Corporate Income Tax|Municipal Surcharge)\b', line_by_line[i]):
                    line_by_line[i] = line_by_line[i].lstrip()
                    line_by_line[i] =  " " + line_by_line[i]
                elif standard_level and white_space_count(line_by_line[i]) != 0: #if it is not zero already
                    line_by_line[i] = remove_leading_whitespace(line_by_line[i], standard_level)
                if re.search(r"^\s*\([a-z]\)", line_by_line[i]):
                    line_by_line[i] = line_by_line[i].strip()

                if re.search(r"\s*%\)", line_by_line[i]):
                    line_by_line[i] = line_by_line[i].strip()
                if re.search(r"^\s*\([p]\)", line_by_line[i]):
                    line_by_line[i] = re.sub(r"^\s*\([p]\)", "(P)", line_by_line[i]) #this make a leading (p) into (P) to not mess up footnotes

            if country_name == "Hungary":
                for i in range(len(line_by_line)):
                    if white_space_count(line_by_line[i]) == 3:
                        line_by_line[i] = re.sub(r'^\s{3}', ' ', line_by_line[i])
                    elif white_space_count(line_by_line[i]) == 4:
                        line_by_line[i] = re.sub(r'^\s{3}', '  ', line_by_line[i])

        #hardcoding Paraguay 2005
        if country_name == "Paraguay" and data_year == "2005":
            line_by_line = line_by_line[1:] #removing the first line
            for i in range(len(line_by_line)):
                removing_extra_numbers = r'(^[^\d]*\d+(?:\.\d+)?(?:/\d+)?)(?:\s+.*)?'
                line_by_line[i] = re.sub(removing_extra_numbers, r"\1", line_by_line[i])

        if country_name == "Ethiopia":
            for i in range(len(line_by_line)):
                line_by_line[i] = re.sub(r'10,000', ' ten thousand', line_by_line[i])
                line_by_line[i] = re.sub(r'500', ' five hundred', line_by_line[i])
        for i in range(len(line_by_line)):
            line_by_line[i] = ''.join(c for c in line_by_line[i] if c.isprintable()) #getting rid of non printable characters
            line_by_line[i] = re.sub(r'(\d)\s+(\d)', r'\1\2', line_by_line[i])

    return(line_by_line, country_name, data_year)

In [173]:
#function to make footnotes
def footnotes_func(line_by_line):
    letter_footnotes_list = []
    current_sentence = ""
    index_letter = [] #list of the letters used for notes

    #get rid of leading white space
    for i in range(len(line_by_line)):
        line_by_line[i] = re.sub(r'\(\s*([a-z])\)', r'(\1)', line_by_line[i])
        
    #this combines lines without a matching letter to the previous line that does
    for line in line_by_line:
        match = re.match(r"^\(([a-z])\)", line)
        if match:
            if current_sentence:
                letter_footnotes_list.append(current_sentence.strip())
            current_sentence = line
        else:
            current_sentence += " " + line

    # Append the last sentence
    if current_sentence:
        letter_footnotes_list.append(current_sentence.strip())

    for i in range(len(letter_footnotes_list)):
        letter_footnotes_list[i] = re.sub(r'\s+', ' ', letter_footnotes_list[i]).strip()

    #skips the first line, this line isn't useful.
    letter_footnotes_list = letter_footnotes_list[1:]

    for line in letter_footnotes_list: # this grabs the letter of the notes
        index_letter.append(line[1])

    letter_footnotes_list = [s[3:] for s in letter_footnotes_list] #this removes the "(a)" out of the text of notes to clean up

    final_footnotes_list = [[index_letter[i], letter_footnotes_list[i]] for i in range(min(len(index_letter), len(letter_footnotes_list)))]


    # footnote_dict = {item[0]: item[1] for item in final_footnotes_list}
    return(final_footnotes_list)


In [270]:
#correcting lines that get cut off to new line
def better_correct_lines(line_by_line):
    ignore = False
    sans_footnotes_list = []
    #making a list without footnotes 
    for line in line_by_line:
        if ignore:
            continue
        if re.search(r'^\s*\((a|u)\).+', line):
            ignore = True
        else:
            sans_footnotes_list.append(line)
        
    final_text_list = []
    
    i = 0
    # Iterate through the list
    while i < len(sans_footnotes_list):
        # Check if the current item matches the footnote pattern
        if re.match(r'^\s*(\([a-z]\))+\s*$', sans_footnotes_list[i]):
            # Extract the footnotes
            all_footnote = sans_footnotes_list[i].strip()
            
            # Append the footnotes to the previous line
            sans_footnotes_list[i - 1] = sans_footnotes_list[i - 1].rstrip() + all_footnote
            
            # Remove the current line containing only footnotes
            sans_footnotes_list.pop(i)
        else:
            # Move to the next item
            i += 1


    #special characters
    symbol_pattern = r'\s{2,}[–—-]\s*\([a-z]\)'  #pattern for special characters (the character with 3 or more leading whitespace followed by a footnote)

    counter = "7777" #starting at 7777, no tax rate will be this high so it should be fine
    special_characters_list = [] #intialzing this list
    for i in range(len(sans_footnotes_list)):
        temp_list = []
        match = re.search(symbol_pattern, sans_footnotes_list[i]) #match for the character
        if match:
            footnote = re.findall(r'\([a-z]\)', sans_footnotes_list[i]) #storing the footnote
                
            temp_list.append(counter) #adding the counter to the temp list
            temp_character = match.group(0) #store the character
            temp_character = temp_character.strip() #removing whitespace
            temp_list.append("") #replace the special character with nothing
            temp_pattern = rf"{re.escape(temp_character)}" #making a pattern for the special character
            special_characters_list.append(temp_list) #saving the list in another list 
            sans_footnotes_list[i] = re.sub(temp_pattern, str(counter), sans_footnotes_list[i]) #replacing the special character with the counter
            if footnote: 
                sans_footnotes_list[i] = sans_footnotes_list[i] + "".join(footnote)
                
            counter = int(counter) + 1 #moving up that counter

    #make Not applicable as zero
    NA_pattern = r'(Not applicable)'
    for i in range(len(sans_footnotes_list)):
        match = re.search(NA_pattern, sans_footnotes_list[i])
        if match:
            sans_footnotes_list[i] = re.sub(NA_pattern, "0", sans_footnotes_list[i])
    
    #this replaces any line with unlimited or indefentite withe 100000
    for i in range(len(sans_footnotes_list)):
        if re.search(r'\b(Unlimited|Indefinite)\b', sans_footnotes_list[i]):
            footnote = re.findall(r'\([a-z]\)', sans_footnotes_list[i]) #storing the footnote
            match = re.search( r'(.+?)\b(Carryforward|Carryback|Corporation Tax)\b', sans_footnotes_list[i])
            if match:
                carry_type = match.group(0)
                sans_footnotes_list[i] = carry_type + " 1000" + "".join(footnote)
            elif re.search(r'Net Operating Losses \(Years\)', sans_footnotes_list[i]):
                sans_footnotes_list[i] = "Net Operating Losses (Years) 1000" + "".join(footnote)
    
    #this replaces any line with "none" with 0
    for i in range(len(sans_footnotes_list)):
        if re.search(r'\bNone\b', sans_footnotes_list[i]):
            footnote = re.findall(r'\([a-z]\)', sans_footnotes_list[i]) #storing the footnote

            match = re.search( r'(.+?)\b(Carryforward|Carryback)\b', sans_footnotes_list[i])
            if match:
                carry_type = match.group(0)
                sans_footnotes_list[i] = carry_type + " 0"
                if footnote: 
                    sans_footnotes_list[i] = sans_footnotes_list[i] + "".join(footnote)
        

    #Take max value for carryforward or carryback
    for i in range(len(sans_footnotes_list)):
        # Extract footnotes
        footnote = re.findall(r'\([a-z]\)', sans_footnotes_list[i])
        
        # Search for match with "Carryforward" or "Carryback"
        match = re.search(r'(.+?)\b(Carryforward|Carryback)\b', sans_footnotes_list[i])
        if match:
            # Extract numbers and find max value
            number_list = [int(num) for num in re.findall(r'\d+', sans_footnotes_list[i])]
            if number_list:
                max_value = max(number_list)
                # Replace the original text with new formatted text
                sans_footnotes_list[i] = match.group(0) + " "+ str(max_value) + "".join(footnote)

    #make any range "5 to 10" to grab max value
    for i in range(len(sans_footnotes_list)):
        footnote = None
        match = re.search(r'(.+?)\b(\d+\s*to\s*\d+)', sans_footnotes_list[i])
        footnote = re.findall(r'\([a-z]\)', sans_footnotes_list[i])  # storing the footnote if exists

        if match:
            number_list = re.findall(r'\d+(?:\.\d+)?', sans_footnotes_list[i])
            if number_list and len(number_list) > 1:
                number_list = [ float(x) for x in number_list ] #turning them into integers
                max_value = str(max(number_list))
                tax_name = match.group(1)
                tax_name = re.sub(r"\d+", "", tax_name)
                sans_footnotes_list[i] = tax_name + max_value + "".join(footnote)
                sans_footnotes_list[i] = re.sub(r'/+', '', sans_footnotes_list[i])

    title_list = []

    for i in range(len(sans_footnotes_list)):
        if re.search(r"Tax Regimes", sans_footnotes_list[i]) and not re.search(r"\d", sans_footnotes_list[i]):
            sans_footnotes_list[i] = re.sub(r"Tax Regimes", "", sans_footnotes_list[i])
    temp_title_list = []
    #seeing what to hard code
    #I run this and output it after looping over all the files, then it's cleaned up.    
    for line in sans_footnotes_list:
        if not re.search(r"\d", line):
            temp_title_list.append(line)

    #list of known titles
    title_list = [
    "Net Operating Losses (Years)",
    "Withholding Tax (%)",
    "Branch Tax Rate (%)",
    "Interest",
    "Dividends",
    "Dividend",
    "Paid to Residents",
    "Royalties",
    "Paid to Nonresidents",
    "Rent",
    "Withholding Tax Rate (%)",
    "Withholding Tax",
    "Rental Income",
    "Sales by Nonresidents",
    "Sales by Residents",
    "Net Operating Losses",
    "Fees for Services",
    "Payments to Residents",
    "Technical Assistance",
    "Sales of Shares",
    "Royalties",
    "Received by Residents",
    "Corporate Income Tax Rate (%)",
    "Capital Gains Tax Rate (%)",
    "Branch Tax Rate (%)",
    "Shareholders’ Loans",
    "Bonds Issued by Companies",
    "Bank Deposits",
    "Payments for Services and Commissions",
    "Corporate Income Tax Rates",
    "From Private Sector Bonds",
    "Professional Fees",
    "Residents",
    "Nonresidents",
    "Payments to Nonresidents",
    "Payments to Resident Individuals",
    "Payments to Resident Companies",
    "Royalties from Patents, Know-how, etc.",
    "Payments under Leases and Rent",
    "Payments to Listed Countries with Preferable Tax Regimes",
    "Lottery Prizes and Other Prize Winnings",
    "Interest on Other Government Securities",
    "Payments for Work Performed by Contractors",
    "Technological Services",
    "Rent of Immovable Property",
    "Rent of Movable Goods",
    "Carryforwards",
    "Payments to Listed Countries with Preferable"]

    #Turning the list of titles into a regex pattern
    title_pattern = re.compile(
        '|'.join(
            r'^\s*' + 
            r'\s*'.join(re.escape(word) for word in title.split()) + 
            r'\s*(?:\([a-z]\))*\s*$' 
            for title in title_list
        ), 
        re.IGNORECASE
    )
    for i in range(len(sans_footnotes_list)):
        # Replace the specific text
        if re.search(r"Payments to Listed Countries with Preferable", sans_footnotes_list[i]):
            sans_footnotes_list[i] = re.sub("Payments to Listed Countries with Preferable", "Payments to Listed Countries with Preferable Tax Regimes", sans_footnotes_list[i])

    #this does multi-line taxes back into one line tax rates
    for i, line in enumerate(sans_footnotes_list):
        # For the first line, add it to the final_text_list directly
        if i == 0:
            final_text_list.append(line)
        else:
            # Check if the previous line contains a numeric character
            if re.search(r'\d', sans_footnotes_list[i-1]):
                # Add the current line directly to the final_text_list
                final_text_list.append(line)
            else:
                # If the previous line matches the title pattern, add the current line separately
                if re.search(title_pattern,(sans_footnotes_list[i - 1])):
                    final_text_list.append(line)
                else:
                    # Concatenate the current line with the previous one
                    final_text_list[-1] = final_text_list[-1] +  " " + line.lstrip()

            
    return(final_text_list, special_characters_list, temp_title_list)

In [198]:
def even_better_title(line_by_line):
    
    titles_exist = False #by default there are no titles at all
    first_title_level = None #same for the leading whitespace of them all
    second_title_level = None
    third_title_level = None
    
    #default none for all titles
    first_title = None 
    second_title = None
    third_title = None

    title_level_list = [] #intializing the list of titles

    #this just checks if there are titles
    for i in range(len(line_by_line)):
        if not re.search(r"\d", line_by_line[i]):
            titles_exist = True
            break

    title_level_list = []
    new_line_by_line = []

    #this is to dynamically find what levels of whitespace of each title 
    if titles_exist: #if titles exists
        for i in range(len(line_by_line)):
            if not re.search(r"\d", line_by_line[i]): #if the line is the same whitespace as the title and non numeric
                title_level_list.append(white_space_count(line_by_line[i]))

        title_level_list = list(set(title_level_list)) #only grab unique levels

        if len(title_level_list) > 0:
            first_title_level = title_level_list[0] #the first item in the list is the first title
        if len(title_level_list) > 1:
            second_title_level = title_level_list[1] #the second item will be the second title
            
        if len(title_level_list) > 2:
            third_title_level = title_level_list[2] #the third item will the third title

        if len(title_level_list) > 3: #if there are more than 3 levels of titles we got a problem
            print("problem in titles lol")

    if titles_exist:
        for i in range(len(line_by_line)):
            if first_title_level is not None: #if there is a first title level, it has to be not none since zero counts as false
                if not re.search(r'\d', line_by_line[i]) and white_space_count(line_by_line[i]) == first_title_level: #does this line equal the same whitespace as a first title
                    first_title = line_by_line[i].strip() #then this is the first title
                    first_title = re.sub(r'\s*\(([a-z])\)', r'(\1)', first_title)

                    
            if second_title_level:
                if not re.search(r'\d', line_by_line[i]) and white_space_count(line_by_line[i]) == second_title_level: #does this line equal the same whitespace as a second title
                    second_title = line_by_line[i].strip() #then this is the second title
                    second_title = re.sub(r'\s*\(([a-z])\)', r'(\1)', second_title)

            if third_title_level:
                if not re.search(r'\d', line_by_line[i]) and white_space_count(line_by_line[i]) == third_title_level: #does this line equal the same whitespace as a third title
                    third_title = line_by_line[i].strip() #then this is a third title
                    third_title = re.sub(r'\s*\(([a-z])\)', r'(\1)', third_title)


            #this is all title logic
            if first_title and not second_title and not third_title: #if there is only a first title, this would usually be above withholding
                if re.search(r'\d', line_by_line[i]) and white_space_count(line_by_line[i]) > first_title_level: #if this line is numeric, add title
                    new_line_by_line.append(first_title + line_by_line[i]) #add the first and second title 
            if second_title and not third_title: #if there is a first title and no third title
                if re.search(r'\d', line_by_line[i]) and white_space_count(line_by_line[i]) > second_title_level: #if this line is more whitespace than the second title
                    new_line_by_line.append(first_title + "(" + second_title + ")"+ line_by_line[i]) #add the first and second title 
            if third_title: #if the third title exists in this iteration, run this
                if re.search(r'\d', line_by_line[i]) and white_space_count(line_by_line[i]) > third_title_level: #if this line has more whitespace than the third title 
                    new_line_by_line.append(first_title  + "(" + second_title + ")"+ "(" + third_title + ")" + line_by_line[i])

            if second_title: #if there is a second title
                if re.search(r"\d", line_by_line[i]) and white_space_count(line_by_line[i]) == second_title_level: #if there is a second title, but this line is on the same as the second title level, only need the first title
                    new_line_by_line.append(first_title + line_by_line[i]) #add the first
                elif re.search(r"\d", line_by_line[i]) and white_space_count(line_by_line[i]) > first_title_level and white_space_count(line_by_line[i]) < second_title_level: #this is for net withholding
                    new_line_by_line.append(first_title + line_by_line[i]) #add the first title
            if second_title and third_title: #if there is a first and second title
                if re.search(r"\d", line_by_line[i]) and white_space_count(line_by_line[i]) == third_title_level: #if a third title exsists, but this line is on the same level as the third title, only need the first and second title
                    new_line_by_line.append(first_title + "(" + second_title + ")"+ line_by_line[i]) #add the first and second title 
            if re.search(r"\d", line_by_line[i]) and white_space_count(line_by_line[i]) == 0: #all lines before the first title
                new_line_by_line.append(line_by_line[i])
    else:
        new_line_by_line = line_by_line


    #this code finds footnotes in the new titled lines and moves to the end like normal footnotes!
    for i in range(len(new_line_by_line)):
        match = re.findall(r'\([a-z]\)', new_line_by_line[i])
        if match:
            new_line_by_line[i] = re.sub('\([a-z]\)', "", new_line_by_line[i])
            for item in match:
                new_line_by_line[i] = new_line_by_line[i] + item


    #this remvoes all extra spaces!
    for i in range(len(new_line_by_line)):
        new_line_by_line[i] = re.sub(r'\s+', ' ', new_line_by_line[i]).strip() #remove extra whitespace and save it
    
    return(new_line_by_line)

In [176]:
#fraction functions
def one_digit_fraction(list):
    sorted_list = list.copy() #copying the list
    sorted_list.sort(key = int)  #sorting it
    #if the rates are in a non linear order there is a fraction in there

    new_value = None
    old_string = None
    target_value = None
    for original_value, sorted_value in zip(list, sorted_list): #if they're not in the same order
        if original_value != sorted_value:
            target_index = list.index(original_value)
            target_value = original_value
            whole_number = int(target_value[0])
            denominator = int(list[target_index + 1])
            numerator = int(target_value[1])
            decimal = round((numerator/denominator), 2)

            new_value = str(whole_number + decimal)
            old_string = str(whole_number) + str(numerator) + "/" + str(denominator)
            break
        else:
            return()

    return(new_value, old_string)

def two_digit_fraction(item):
    #if there is a 3 digit tax rate
    whole_number = int(item[:2])
    numerator = int(item[-1])
    denominator = int(item[1])

    decimal = round((numerator/denominator), 2)

    new_value = str(whole_number + decimal)
    old_value = str(whole_number) + str(numerator) + "/" + str(denominator)
    return(new_value, old_value)

In [177]:
#dealing with multi rates
def multi_rate_func(line_by_line):
    multi_pattern = re.compile(r'\b(\d+(?:\.\d+)?(?:/\d+(?:\.\d+)?)+)\b') # pattern to catch multi rates
    multi_line_list = []
    unique_counter = 9100 #starting the unique number, ideally no taxes are this high so this shouldn't interer with real rates

    #save the multi rate lines 
    for i in range(len(line_by_line)):
        multi_lines = re.search(multi_pattern, line_by_line[i])
        if multi_lines:
            temp_list = []
            temp_list.append(unique_counter)
            temp_list.append(multi_lines.group(0))
            line_by_line[i] = re.sub(multi_pattern, (str(unique_counter)), line_by_line[i])
            unique_counter = unique_counter + 1
            multi_line_list.append(temp_list)
                
    all_under_two = False
    #all of this is to deal with fractions
    if multi_line_list:
        for sublist in multi_line_list:
            split_list = sublist[1].split("/")
            for i in range(len(split_list)):
                if "." in split_list[i]:
                    break
                if re.findall(r'\d{3,}', split_list[i]):
                    new_value, old_string = two_digit_fraction(split_list[i])
                    sublist[1] = sublist[1].replace(old_string, new_value)

            for i in range(len(split_list)):
                if len(split_list[i]) >= 2:
                    all_under_two = False
                    break

            if all_under_two:
                new_value, old_string = one_digit_fraction(split_list)
                sublist[1] = sublist[1].replace(old_string, new_value)
    
                
    return(line_by_line, multi_line_list)


In [254]:
def max_multi_rate_function(line_by_line):
    # pattern = re.compile(r'(.+?)\s+((?:\d+/\d+(?:/\d+)*)|\d+/[0-9]+)\s*((?:\([a-z]\))*)\s*$', re.IGNORECASE) #pattern for multi rates, such as "10/15/20"
    pattern = re.compile(r'(.+?)\s+((?:\d+(?:\.\d+)?/\d+(?:\.\d+)?(?:/\d+(?:\.\d+)?)*))\s*((?:\([a-z]\))*)\s*$', re.IGNORECASE) #great one
    # pattern = re.compile(r'(.+?)\s+((?:\d+(?:\.\d+)?(?:/\d+(?:\.\d+)?)*(?:\s*\d+(?:\.\d+)?)*(?:/\d+(?:\.\d+)?)*))\s*((?:\([a-z]\))*)\s*$', re.IGNORECASE) #new one

    all_under_two = True
    footnotes = None


    
    # Iterate over each line and process
    for i in range(len(line_by_line)):

        match = pattern.search(line_by_line[i])
        if match:
            #split things into the three elements 
            tax_rate_line = match.group(1)
            multi_rate_line = match.group(2).replace(" ", "")
            split_list = multi_rate_line.split("/")
            footnotes = re.findall(r'\([a-z]\)', line_by_line[i])
            # Example logic to handle fractions
            for j in range(len(split_list)):
                if "." in split_list[j]:
                    break
                if re.findall(r'\d{3,}', split_list[j]):
                    # Assuming two_digit_fraction function changes format
                    new_value, old_string = two_digit_fraction(split_list[j])
                    multi_rate_line = multi_rate_line.replace(old_string, new_value)

                for k in range(len(split_list)):
                    if len(split_list[k]) >= 2:
                        all_under_two = False
                        break

                if all_under_two:
                    # Assuming one_digit_fraction function changes format
                    if one_digit_fraction(split_list): #if there is any fractions to deal with, if anything is returned, then run the function
                        new_value, old_string = one_digit_fraction(split_list)
                        multi_rate_line = multi_rate_line.replace(old_string, new_value)

            if "/" in line_by_line[i]: #if there is a multi rate
                max_value_list = multi_rate_line.split("/") #split them by "/"
                max_value_list = [ float(x) for x in max_value_list ] #turning them into integers

                max_value = max(max_value_list) #grab the highest value
                line_by_line[i] = f"{tax_rate_line}       {str(max_value)}"
            else:
                line_by_line[i] = f"{tax_rate_line}       {multi_rate_line}" #this will return a line, if it used to be a fraction.

            if footnotes:
                line_by_line[i] = line_by_line[i] + "".join(footnotes) #add the list of footnotes.

    multi_line_list = [] #making the list empty to not run in the final function
    return(line_by_line, multi_line_list)

In [180]:
def better_split_lines(line_by_line):
    final_list = []

    seen = set()
    duplicates = set()
    remove_index = []

    for i in range(len(line_by_line)):
        if line_by_line[i] in seen:
            duplicates.add(line_by_line[i])
            remove_index.append(i)
        else:
            seen.add(line_by_line[i])

    if duplicates:
        for index in remove_index:
            line_by_line.pop(index)

    for line in line_by_line:
        pattern = r'^(.*?)(\d[\d./]*)(?:\s*(\([a-z]\))*)\s*$' #regex for things before numberic lines, the numeric lines and the footnotes
        
        # Attempt to match the pattern in the text
        match = re.match(pattern, line, re.IGNORECASE)
        if match:
            # Extract components
            tax_rate = match.group(1).strip()  # Tax rate part
            numeric_value = match.group(2)      # Numeric value part
            footnotes = match.group(3)         # Footnotes (optional)
            # Format the output as required
            final_lines = [tax_rate, numeric_value]
            
            if footnotes:
                footnotes = re.findall(r'\(([a-z])\)', line) #if there are footnotes, grab them as a list
            else:
                footnotes = []

            final_lines = [tax_rate, numeric_value] + footnotes #add footnotes to the list
            final_list.append(final_lines) #add list to the the list of all the things
        
    return(final_list)

In [181]:
#final cleaning and replacing temp values
def final_output(final_lines, multi_line_list, special_characters_list):

    #here I'd like to replace the 9100's with their actual numbers
    if multi_line_list:
        for i, final_element in enumerate(final_lines):
            for unique_element in multi_line_list:
                if final_element[1] == str(unique_element[0]):
                    final_lines[i][1] = unique_element[1]
                

    #replace special characters withe a blank
    if special_characters_list: 
        for i, final_element, in enumerate(final_lines):
            for unique_element in special_characters_list:
                if final_element[1] == str(unique_element[0]):
                    final_lines[i][1] = unique_element[1]
    
    # if final_footnotes_list:
    #     for sublist in (final_footnotes_list):

    #         footnote = sublist[0]
    #         footnote_text = sublist[1]
            
    #         for k in range(len(final_lines)):
    #             sublist = final_lines[k]

    #             for j in range(len(sublist)):
    #                 if footnote == sublist[j]:
    #                     sublist[j] = footnote_text    
    
    final_list = final_lines

    return(final_list)

In [182]:
#check outputs function
def check_output(final_list, line_by_line,final_footnotes_list, country_name, data_year):
    problems = []
    final_corporate = False
    has_corporate = False

    for line in line_by_line:
        if re.search(r'Corporate(?:\s+\w+)+\s+Tax\s+Rate(?:\s*\(%\))?', line):
            has_corporate = True       
    
    # if final_footnotes_list:
    #     # Extract elements after the first two from each sublist in final_list
    #     flat_list = [item for sublist in final_list for item in sublist[2:]]
    #     # Remove duplicates by converting to set and back to list
    #     unique_flat_list = list(set(flat_list))
    #     # Check if any elements in final_footnotes_list are missing in unique_flat_list
    #     missing_elements = set()
    #     for sublist in final_footnotes_list:
    #         missing_elements.update(set(sublist[2]) - set(unique_flat_list))

    #     if missing_elements:
    #         problems.append("there is an orphaned footnote")
    #         problems.append(missing_elements)

    for line in final_list:
        if re.search(r"\([a-z]\)", line[0]):
            problems.append("there is a footnotes in the tax name")
        if re.search(r"\([a-z]\)", line[1]):
            problems.append("there is a footnotes in the tax rate")
        if line[1].isalpha():
            if len(line[1]) > 11:
                problems.append("tax rate might be too long")
        if re.search(r'Corporate(?:\s+\w+)+\s+Tax\s+Rate(?:\s*\(%\))?', line[0]):
            final_corporate = True
            break
        if re.search(r'[^a-zA-Z0-9.–/ ]', line[1]):
            problems.append("Any non letters or numbers (excluding decimals)")
            #print(line[1], country_name, data_year, "non numeric characters")
        if line[0].isnumeric():
            problems.append("number in the tax name")
            #print(line[0], country_name, data_year, "number in tax name")
    
    tax_names = [sublist[0] for sublist in final_list]

    seen = set()
    duplicates = set()

    for item in tax_names:
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)

    if duplicates:
        problems.append("there are duplicates")
        for duplicate in duplicates:
            print(duplicate)

    if final_corporate is False and has_corporate is True:
        problems.append("has corporate tax rate, then loses it")

    
    return(problems)

In [183]:
def dataframe_function(final_list, country, year, final_footnotes_list):
    letter_columns = list(string.ascii_lowercase[:21])  # predefined letter columns from 'a' to 'u'
    columns = ["Country", "Year", "Tax Name", "Tax rate"] + letter_columns

    rows = []

    footnotes_dict = {item[0]: item[1] for item in final_footnotes_list}

    for row in final_list:
        row_data = {
            "Country": country,
            "Year": year,
            "Tax Name": row[0],
            "Tax rate": row[1]
        }
        #this turns the place holding letters as footnotes
        for letter in letter_columns:
            # Check if the letter is in the row and replace with footnote text if applicable
            if letter in row[2:]:
                row_data[letter] = footnotes_dict.get(letter, letter)
            else:
                row_data[letter] = ""
        # to leave the place holding letters
        # for letter in letter_columns:
        #     row_data[letter] = letter if letter in row[2:] else ""
        rows.append(row_data)
        
    return rows, columns

In [274]:
working_directory = directory +  "/global_taxation/Code/trying_git/tax_parsing/Outputs/Philippines/text_files/Philippines_2019.txt"

all_rows = []
all_columns = ["Country", "Year", "Tax Name", "Tax rate"] + list(string.ascii_lowercase)


line_by_line, country_name, data_year = get_table(working_directory)
# for line in line_by_line: print(line)
final_footnotes_list = footnotes_func(line_by_line)
# for line in final_footnotes_list: print(line)

# for line in final_footnotes_list: print(line)

line_by_line, special_characters_list, temp_title_list = better_correct_lines(line_by_line)
# for line in line_by_line: print(line)

line_by_line, multi_line_list = max_multi_rate_function(line_by_line)
# for line in line_by_line: print(line)

line_by_line = even_better_title(line_by_line)
# for line in line_by_line: print(line)


final_list  = better_split_lines(line_by_line)
# for line in final_list: print(line)

final_list = final_output(final_list,multi_line_list,special_characters_list)
# temp_problem_list = check_output(final_list, line_by_line, final_footnotes_list, country_name, data_year)
for line in final_list: print(line)
# for line in temp_problem_list: print(line)
# # for line in final_footnotes_list: print(line)

# rows, columns = dataframe_function(final_list, country_name, data_year,final_footnotes_list)
# all_rows.extend(rows)

# # print(all_rows)
# df = pd.DataFrame(all_rows, columns=all_columns)

# df

['Corporate Income Tax Rate (%)', '30']
['Capital Gains Tax Rate (%) Real Property', '6', 'a']
['Capital Gains Tax Rate (%) Shares Sellers Who are Citizens, Resident Aliens or Domestic Corporations', '15', 'b', 'c']
['Capital Gains Tax Rate (%) Other Sellers', '10.0', 'd']
['Branch Income Tax Rate', '30', 'e']
['Withholding Tax(Dividends) Nonresident Foreign Corporations', '30.0', 'f']
['Withholding Tax(Dividends) Domestic Corporations and Resident Foreign Corporations', '0', 'g']
['Withholding Tax Interest on Peso Deposits', '20', 'h', 'i']
['Withholding Tax Royalties from Patents, Know-how, etc.', '20', 'i']
['Withholding Tax Branch Profits Remittance Tax', '15']
['Net Operating Losses (Years) Carryback', '0']
['Net Operating Losses (Years) Carryforward', '3', 'j']


In [211]:

# Sample text
text = "Branch Profits Tax Rate (Additional Tax) (%)                            — (d)"

# More flexible regex pattern
pattern = r'\s{2,}[–—-]\s*\([a-z]\)'

match = re.search(pattern, text)
if match:
    print("Match found:", match.group())
else:
    print("No match found.")


Match found:                             — (d)


In [271]:
#here is the loop
# root_dir = directory + "global_taxation/Code/trying_git/tax_parsing/temp_outputs"
root_dir = directory + "global_taxation/Code/trying_git/tax_parsing/Outputs"

log_list = []
potential_titles = []
all_rows = []
all_columns = ["Country", "Year", "Tax Name", "Tax rate"] + list(string.ascii_lowercase[:21])

#this is to empty the file before runnning the loop
with open("log_file.txt", "w") as file:
    pass

#this is the loop to go through every file in the directory
for root, dirs, files in os.walk(root_dir):
    # Check if the current folder is the "Outputs" folder
    if os.path.basename(root) == "Outputs":
        # Iterate over the country folders inside "Outputs"
        for country_dir in dirs:
            # Construct the path to the "text_files" folder inside the current country folder
            text_files_dir = os.path.join(root, country_dir, "text_files")
            # Check if the "text_files" folder exists
            if os.path.isdir(text_files_dir):
                # Loop through the text files inside the "text_files" folder
                for file in os.listdir(text_files_dir):
                    file_path = os.path.join(text_files_dir, file)
                    # Process each text file as needed
                    problems = []
                    
                    try:
                        line_by_line, country_name, data_year = get_table(file_path)
                        try:
                            final_footnotes_list = footnotes_func(line_by_line)
                        except:
                            problems.append("something wrong with footnotes function")

                        try:
                            line_by_line, special_characters_list, temp_title_list = better_correct_lines(line_by_line)
                            potential_titles.extend(temp_title_list)
                        except:
                            problems.append("something wrong with correcting name lengths function")

                        try:
                            line_by_line, unique_multi_list = max_multi_rate_function(line_by_line)
                        except:
                            problems.append("something wrong with multi lines function")

                        try:
                            line_by_line = even_better_title(line_by_line)
                        except:
                            problems.append("something adding titles function")

                        try:
                            final_list  = better_split_lines(line_by_line)
                        except:
                            problems.append("something wrong with seperating the list elements into columns function")

                        try:
                            final_list = final_output(final_list,unique_multi_list,special_characters_list)
                            
                        except:
                            problems.append("something wrong with the final output function")

                        try:
                            temp_problem_list = check_output(final_list, line_by_line,final_footnotes_list, country_name, data_year)
                            problems = problems + temp_problem_list
                        except:
                            problems.append("the checking function doesn't work somehow")

                        try:
                            rows, columns = dataframe_function(final_list, country_name, data_year, final_footnotes_list)
                            all_rows.extend(rows)
                        except:
                            problems.append("the dataframe function")
                    except:
                        #because there might be an error in the function, it won't output country name and data year correctly, so defining it outside of the loop here
                        country_name = file[:-9] #Name of the country
                        data_year = file[-8:-4] #What year is the data
                        problems.append("something wrong with the table function")


                    log_dictionary = {"country": country_name, 
                                    "year": data_year,
                                    "problem count": len(problems),
                                    "problems": problems}
                    
                    if len(problems) != 0:
                        with open("log_file.txt", "a") as file:
                            file.write(f"{country_name}, {data_year}, problem count: {len(problems)}, problems: {problems}\n")

                    log_list.append(log_dictionary)

#this is to count how many files have "zero" issues
zero_count = 0                 
for dictionary in log_list:
    # Get the "problem count" value from each dictionary
    problem_count = dictionary["problem count"]
    # Count zeros
    if problem_count == 0:
        zero_count += 1            

# Ensure all expected columns are in the DataFrame
df = pd.DataFrame(all_rows, columns=all_columns)


print(zero_count/len(log_list) * 100)
print(len(log_list)- zero_count, "cases left")


df.to_csv(directory + "global_taxation/Code/trying_git/tax_parsing/full_file.csv", index = False)

missing table Laos 2007
99.96790757381258
1 cases left


In [48]:

#here is a thing to grab what might be titles and then cleaned up
better_titles_list = []
temp_list = []
for i in range(len(potential_titles)):
    if re.search(r'\([a-z]\)$', potential_titles[i]):
        potential_titles[i] = re.sub(r'\([a-z]\)', "", potential_titles[i])
        temp_list.append(potential_titles[i])

for i in range(len(potential_titles)):
    potential_titles[i] = re.sub(r'^[\s\u2000-\u200b\u3000]+|[\s\u2000-\u200b\u3000]+$', "", potential_titles[i]) #get rid of whitespace
    potential_titles[i] = re.sub(r'\([a-z]\)', "", potential_titles[i])
    if not re.search(r'(?:and|,|a|or|of|to|in|-)$', potential_titles[i]) and re.search(r'^[A-Z]', potential_titles[i]): #removing anything ending in "-" as it is likely a new line sentance
        better_titles_list.append(potential_titles[i])

better_titles_list.extend(temp_list)


from collections import Counter

def find_most_frequent_duplicates(items):
    # Count the occurrences of each item in the list
    item_counts = Counter(items)
    
    # Filter out items that are not duplicates (i.e., have a count of 1)
    duplicates = {item: count for item, count in item_counts.items() if count > 1}
    
    # Sort the duplicates by their frequency in descending order
    sorted_duplicates = sorted(duplicates.items(), key=lambda x: x[1], reverse=True)
    
    # Convert to a list of lists, where each sublist contains the item and its frequency
    result = [[item, count] for item, count in sorted_duplicates]
    
    return result

most_frequent_duplicates = find_most_frequent_duplicates(better_titles_list)
print(len(most_frequent_duplicates))
for item, frequency in most_frequent_duplicates:
    print(f"{frequency}\t{item}")



In [ ]:
#loop to keep multi rates

#here is the loop
root_dir = directory + "global_taxation/Code/trying_git/tax_parsing/Outputs"
log_list = []
potential_titles = []
all_rows = []
all_columns = ["Country", "Year", "Tax Name", "Tax rate"] + list(string.ascii_lowercase[:21])

#this is to empty the file before runnning the loop
with open("log_file.txt", "w") as file:
    pass

#this is the loop to go through every file in the directory
for root, dirs, files in os.walk(root_dir):
    # Check if the current folder is the "Outputs" folder
    if os.path.basename(root) == "Outputs":
        # Iterate over the country folders inside "Outputs"
        for country_dir in dirs:
            # Construct the path to the "text_files" folder inside the current country folder
            text_files_dir = os.path.join(root, country_dir, "text_files")
            # Check if the "text_files" folder exists
            if os.path.isdir(text_files_dir):
                # Loop through the text files inside the "text_files" folder
                for file in os.listdir(text_files_dir):
                    file_path = os.path.join(text_files_dir, file)
                    # Process each text file as needed
                    problems = []
                    
                    try:
                        line_by_line, country_name, data_year = get_table(file_path)
                        try:
                            final_footnotes_list = footnotes_func(line_by_line)
                        except:
                            problems.append("something wrong with footnotes function")

                        try:
                            line_by_line, special_characters_list, temp_title_list = better_correct_lines(line_by_line)
                            potential_titles.extend(temp_title_list)
                        except:
                            problems.append("something wrong with correcting name lengths function")

                        try:
                            line_by_line, unique_multi_list = multi_rate_func(line_by_line)
                        except:
                            problems.append("something wrong with multi lines function")

                        try:
                            line_by_line = even_better_title(line_by_line)
                        except:
                            problems.append("something adding titles function")

                        try:
                            final_list  = better_split_lines(line_by_line)
                        except:
                            problems.append("something wrong with seperating the list elements into columns function")

                        try:
                            final_list = final_output(final_list,unique_multi_list,special_characters_list)
                            
                        except:
                            problems.append("something wrong with the final output function")

                        try:
                            temp_problem_list = check_output(final_list, line_by_line,final_footnotes_list, country_name, data_year)
                            problems = problems + temp_problem_list
                        except:
                            problems.append("the checking function doesn't work somehow")

                        try:
                            rows, columns = dataframe_function(final_list, country_name, data_year, final_footnotes_list)
                            all_rows.extend(rows)
                        except:
                            problems.append("the dataframe function")
                    except:
                        #because there might be an error in the function, it won't output country name and data year correctly, so defining it outside of the loop here
                        country_name = file[:-9] #Name of the country
                        data_year = file[-8:-4] #What year is the data
                        problems.append("something wrong with the table function")


                    log_dictionary = {"country": country_name, 
                                    "year": data_year,
                                    "problem count": len(problems),
                                    "problems": problems}
                    
                    if len(problems) != 0:
                        with open("log_file.txt", "a") as file:
                            file.write(f"{country_name}, {data_year}, problem count: {len(problems)}, problems: {problems}\n")

                    log_list.append(log_dictionary)

#this is to count how many files have "zero" issues
zero_count = 0                 
for dictionary in log_list:
    # Get the "problem count" value from each dictionary
    problem_count = dictionary["problem count"]
    # Count zeros
    if problem_count == 0:
        zero_count += 1            

# Ensure all expected columns are in the DataFrame
df = pd.DataFrame(all_rows, columns=all_columns)


print(zero_count/len(log_list) * 100)
print(len(log_list)- zero_count, "cases left")


df.to_csv(directory + "global_taxation/Code/trying_git/tax_parsing/full_file_multi_rate.csv", index = False)